In [1]:
import os
import numpy as np
import pandas as pd

from scipy import stats
from scipy.ndimage import gaussian_filter
from scipy.signal import find_peaks

import pingouin

import matplotlib as mpl
import matplotlib.pyplot as plt  
import seaborn as sns

from datetime import date
import random

In [2]:
# no top and right spines in all plots
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

In [3]:
mother_path = 'D:/Multi-modal project/'
save_path = mother_path+'analysis/result/4.1. Surrogate correlations/'

### Parameter setting

In [4]:
sig_alpha = 0.01
sig_cohend = 0.4
cons_bin_crit = 5

gauss_sigma = 2
gauss_on = True

only_PER = False   # analyze only PER neurons?

condition = ['Multimodal','Visual','Auditory','Control']
 # colors for multimodal, vis-only, aud-only conditions
colorpalette = ['mediumorchid','cornflowerblue','lightcoral','gray']

cat = ['M','V','A','MV','MA','VA','MVA','None']

today = str(date.today())

### Data preparation

In [5]:
cell_path = mother_path+'analysis/result/zFR export/13-Apr-2022 (5 trials)/'

data_path = mother_path+'analysis/result/3. Item-selectivity/2023-01-09/'
df = pd.read_csv(data_path+'2023-01-09_item-selectivity.csv')
os.chdir(data_path)

# analyze only PER neurons
if only_PER == True:
    df = df[df['Region']=='PER']
    df.reset_index(inplace=True,drop=True)
df.insert(4,'Cat','')

for i in range(len(df)):
    cat_info = ''
    if df.loc[i,'M_sig'] == 1:
        cat_info += 'M'
    if df.loc[i,'V_sig'] == 1:
        cat_info += 'V'
    if df.loc[i,'A_sig'] == 1:
        cat_info += 'A'
    if cat_info=='':
        df.loc[i,'Cat'] = 'None'
    else:
        df.loc[i,'Cat'] = cat_info

In [6]:
os.chdir(data_path)
Binary_M = pd.read_csv('Binary_M.csv')
Binary_V = pd.read_csv('Binary_V.csv')
Binary_A = pd.read_csv('Binary_A.csv')
Binary_C = pd.read_csv('Binary_C.csv')

Cohend_M = pd.read_csv('Cohend_M.csv')
Cohend_V = pd.read_csv('Cohend_V.csv')
Cohend_A = pd.read_csv('Cohend_A.csv')
Cohend_C = pd.read_csv('Cohend_C.csv')

In [7]:
# empty list for result csv file
result=[]

### Surrogate correlations
to get surrogate distribution of object-selective correlations by randomly selecting M or V or A cells and calculating correlations between them.

In [126]:
def shuffle_and_corr(cond):
    if cond=='MV':
        r1 = random.choice(df[df.Cat=='M'].Key.values)
        r2 = random.choice(df[df.Cat=='V'].Key.values)
        
        b1 = Binary_M.loc[Binary_M.iloc[:,0]==r1].iloc[:,1:].values[0]
        b2 = Binary_V.loc[Binary_V.iloc[:,0]==r2].iloc[:,1:].values[0]
        
        c1 = Cohend_M.loc[Cohend_M.iloc[:,0]==r1].iloc[:,2:].values[0]
        c2 = Cohend_V.loc[Cohend_V.iloc[:,0]==r2].iloc[:,2:].values[0]
    elif cond=='MA':
        r1 = random.choice(df[df.Cat=='M'].Key.values)
        r2 = random.choice(df[df.Cat=='A'].Key.values)
        
        b1 = Binary_M.loc[Binary_M.iloc[:,0]==r1].iloc[:,1:].values[0]
        b2 = Binary_A.loc[Binary_A.iloc[:,0]==r2].iloc[:,1:].values[0]
        
        c1 = Cohend_M.loc[Cohend_M.iloc[:,0]==r1].iloc[:,2:].values[0]
        c2 = Cohend_A.loc[Cohend_A.iloc[:,0]==r2].iloc[:,2:].values[0]
    elif cond=='VA':
        r1 = random.choice(df[df.Cat=='V'].Key.values)
        r2 = random.choice(df[df.Cat=='A'].Key.values)
        
        b1 = Binary_V.loc[Binary_V.iloc[:,0]==r1].iloc[:,1:].values[0]
        b2 = Binary_A.loc[Binary_A.iloc[:,0]==r2].iloc[:,1:].values[0]
        
        c1 = Cohend_V.loc[Cohend_V.iloc[:,0]==r1].iloc[:,2:].values[0]
        c2 = Cohend_A.loc[Cohend_A.iloc[:,0]==r2].iloc[:,2:].values[0]
        
    c1[b1==0] = 0
    c2[b2==0] = 0
    
    result = stats.kendalltau(c1,c2)[0]
    #result = stats.pearsonr(c1,c2)[0]
    
    return result

In [127]:
%%time
flag = 1
count = 0
while flag == 1:
    num_shuffle = 1000
    surr_corr = np.zeros((num_shuffle,3))

    for i in range(num_shuffle):
        surr_corr[i,0] = shuffle_and_corr('MV')
        surr_corr[i,1] = shuffle_and_corr('MA')
        surr_corr[i,2] = shuffle_and_corr('VA')
        
    corr_crit = [np.percentile(surr_corr[:,0],95),
                 np.percentile(surr_corr[:,1],95),
                 np.percentile(surr_corr[:,2],95)]
    
    if max(corr_crit)<0.48:
        flag = 0
        
    count +=1
    print(f'shuffle run = {count} / M-V = {corr_crit[0]:.2f} / M-A = {corr_crit[1]:.2f} / V-A = {corr_crit[2]:.2f}')

shuffle run = 1 / M-V = 0.50 / M-A = 0.51 / V-A = 0.50
shuffle run = 2 / M-V = 0.47 / M-A = 0.53 / V-A = 0.54
shuffle run = 3 / M-V = 0.51 / M-A = 0.48 / V-A = 0.50
shuffle run = 4 / M-V = 0.55 / M-A = 0.51 / V-A = 0.52
shuffle run = 5 / M-V = 0.51 / M-A = 0.53 / V-A = 0.48
shuffle run = 6 / M-V = 0.54 / M-A = 0.51 / V-A = 0.48
shuffle run = 7 / M-V = 0.50 / M-A = 0.47 / V-A = 0.48
shuffle run = 8 / M-V = 0.55 / M-A = 0.50 / V-A = 0.48
shuffle run = 9 / M-V = 0.49 / M-A = 0.48 / V-A = 0.51
shuffle run = 10 / M-V = 0.51 / M-A = 0.49 / V-A = 0.46
shuffle run = 11 / M-V = 0.48 / M-A = 0.52 / V-A = 0.44
shuffle run = 12 / M-V = 0.44 / M-A = 0.48 / V-A = 0.50
shuffle run = 13 / M-V = 0.51 / M-A = 0.53 / V-A = 0.48
shuffle run = 14 / M-V = 0.51 / M-A = 0.46 / V-A = 0.45
shuffle run = 15 / M-V = 0.49 / M-A = 0.52 / V-A = 0.51
shuffle run = 16 / M-V = 0.45 / M-A = 0.54 / V-A = 0.52
shuffle run = 17 / M-V = 0.49 / M-A = 0.52 / V-A = 0.47
shuffle run = 18 / M-V = 0.44 / M-A = 0.58 / V-A = 0.50
s

In [128]:
# display the significant level of correlation
print('M-V correlation (>95%) = ' +str(np.percentile(surr_corr[:,0],95)))
print('M-A correlation (>95%) = ' +str(np.percentile(surr_corr[:,1],95)))
print('V-A correlation (>95%) = ' +str(np.percentile(surr_corr[:,2],95)))

M-V correlation (>95%) = 0.4732391598646934
M-A correlation (>95%) = 0.4746449088336034
V-A correlation (>95%) = 0.4776058356554091


In [95]:
surr_corr = pd.DataFrame(surr_corr)
surr_corr.rename(columns={0:'M-V corr',1:'M-A corr',2:'V-A corr'},inplace=True)
surr_corr.head(5)

,M-V corr,M-A corr,V-A corr
0,-0.506775,0.001919,0.373951
1,-0.132492,0.142833,-0.206475
2,0.131790,-0.083933,0.130423
3,0.145863,-0.069475,0.242224
4,-0.191463,0.079659,-0.097300


In [11]:
# export surrogate correlations to CSV
if os.path.exists(save_path+str(date.today())) is False:
    os.makedirs(save_path+str(date.today()))    

surr_corr.to_csv(save_path+str(date.today())+'/abc.csv',index=False)